In [1]:
# !pip install transformers
!pip install bitsandbytes -q

import torch
from transformers import M2M100ForConditionalGeneration
from transformers import AutoTokenizer
from transformers.modeling_outputs import BaseModelOutput
from huggingface_hub import snapshot_download
from transformers import SiglipImageProcessor
from datasets import load_dataset
from huggingface_hub import login
from PIL import Image
import torch

# model_name = "mtmlt/sonar-nllb-200-1.3B"
# model_dir = snapshot_download(model_name)

# model = M2M100ForConditionalGeneration.from_pretrained(model_dir)
# tokenizer = AutoTokenizer.from_pretrained(model_dir)

# def encode_mean_pool(texts, tokenizer, encoder, lang='eng_Latn', norm=False):
#     tokenizer.src_lang = lang
#     with torch.inference_mode():
#         batch = tokenizer(texts, return_tensors='pt', padding=True)
#         seq_embs = encoder(**batch)
#         mask = batch.attention_mask
#         mean_emb = (seq_embs.last_hidden_state * mask.unsqueeze(-1)).sum(1) / mask.unsqueeze(-1).sum(1)
#         if norm:
#             mean_emb = torch.nn.functional.normalize(mean_emb)
#     return mean_emb, seq_embs, mask

# sentences = ['My name is SONAR.', 'I can embed the sentences into vector space.']

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour 

2025-10-28 16:58:52.711603: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761670732.898865      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761670732.952544      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# train_adapter_align.py
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import M2M100ForConditionalGeneration, AutoTokenizer, SiglipImageProcessor
from transformers.modeling_outputs import BaseModelOutput
from huggingface_hub import snapshot_download
from tqdm import tqdm
import os

model_name = "mtmlt/sonar-nllb-200-1.3B"
model_dir = snapshot_download(model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"

decoder_model = M2M100ForConditionalGeneration.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(model_dir)
# Ensure target language is English Latin for captioning
tokenizer.tgt_lang = 'eng_Latn'
tokenizer.src_lang = 'eng_Latn'

img_model_dir = snapshot_download("google/siglip-base-patch16-384")
image_processor = SiglipImageProcessor.from_pretrained(img_model_dir)  

decoder_model.eval()  

decoder_encoder = decoder_model.model.encoder  
decoder_model.eval()

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

.gitattributes: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

README.md: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/804 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/409 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/711 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/814M [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

M2M100ForConditionalGeneration(
  (model): M2M100Model(
    (shared): M2M100ScaledWordEmbedding(256206, 1024, padding_idx=1)
    (encoder): M2M100Encoder(
      (embed_tokens): M2M100ScaledWordEmbedding(256206, 1024, padding_idx=1)
      (embed_positions): M2M100SinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0-23): 24 x M2M100EncoderLayer(
          (self_attn): M2M100Attention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=8192, bias=True)
          (fc2): Linear(in_features=8192, out_features=1024, bias=True)
       

In [3]:
# Configuration flags
USE_VISUAL_PREFIX = False  # Set True to use visual prefix tokens; False to use single-token image projection
VISUAL_SEQ_LEN = 16        # Only used when USE_VISUAL_PREFIX=True

In [4]:
import torch.nn as nn

import torch
import torch.nn as nn
from transformers import SiglipVisionModel

class SonarImageEnc(nn.Module):
    def __init__(self, path="google/siglip-base-patch16-384", initial_temperature=0.07):
        super().__init__()
        
        self.register_buffer("one", torch.tensor(1.0))
        self.register_buffer("min_temp", torch.tensor(0.01))
        self.register_buffer("max_temp", torch.tensor(100.0))
        
        self.model = SiglipVisionModel.from_pretrained(path, torch_dtype="auto")

        for param in self.model.parameters():
            param.requires_grad = False

        self.lang_projection = nn.Linear(2048, 1024)

        self.projection = nn.Sequential(
            nn.Linear(self.model.config.hidden_size, 4096),
            nn.GELU(),
            nn.Dropout(0.1),
            nn.Linear(4096, 2048),
            nn.LayerNorm(2048, eps=1e-6),
            nn.GELU(),
            nn.Dropout(0.3),
            nn.Linear(2048, 2048),
            nn.GELU(),
            nn.Dropout(0.3),
            nn.Linear(2048, 1024)
        )

        for param in self.projection.parameters():
            param.requires_grad = True

        self.temp_s = nn.Parameter(torch.log(torch.tensor(10.0)))
        self.bias = nn.Parameter(torch.tensor(-10.0))
        self.logit_scale = nn.Parameter(
            torch.ones([]) * torch.log(torch.tensor(1.0) / initial_temperature)
        )

    def forward(self, pixel_values, lang_token=None):
        device = pixel_values.device

        vision_outputs = self.model(pixel_values=pixel_values)
        pooled_output = vision_outputs.pooler_output 

        x = self.projection[0:3](pooled_output)
        x = self.projection[3:6](x)
        residual = x
        x = self.projection[6:9](x)
        x = x + residual
        x = self.projection[9](x)
        embeddings = self.projection[10](x) 

        combined_embs = None
        if lang_token is not None:
            lang_token_expanded = lang_token.expand(embeddings.size(0), -1, -1).to(device)

            image_emb_pred = embeddings
            combined_embs = torch.cat([lang_token_expanded, image_emb_pred], dim=1)
        else:
            combined_embs = embeddings

        scale = torch.exp(self.logit_scale).clamp(min=1.0/self.max_temp, max=1.0/self.min_temp)

        return combined_embs, scale, torch.exp(self.temp_s), self.bias


image_encoder = SonarImageEnc(path=img_model_dir).to(device)

In [5]:
import os

ckpt_path = "/kaggle/input/sonar/transformers/default/1/best_model_step4000.pt"
if os.path.exists(ckpt_path):
    state = torch.load(ckpt_path, weights_only=True)
    image_encoder.load_state_dict(state, strict=False)
    print(f"Loaded adapter checkpoint from {ckpt_path}")
else:
    print("No external adapter checkpoint found; training from scratch.")

Loaded adapter checkpoint from /kaggle/input/sonar/transformers/default/1/best_model_step4000.pt


In [6]:
def text_to_sonar_embedding(texts=None, tokenizer=None, encoder_model=None, batch=None, lang='eng_Latn', device='cpu', norm=False):
    if batch is None:
        tokenizer.src_lang = lang
        batch = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
    batch = {k: v.to(device) for k, v in batch.items()}
    encoder_model.to(device)
    with torch.no_grad():
        seq_embs = encoder_model(**batch)
    mask = batch['attention_mask']
    mean_emb = (seq_embs.last_hidden_state * mask.unsqueeze(-1)).sum(1) / mask.sum(1, keepdim=True)
    if norm:
        mean_emb = F.normalize(mean_emb, dim=-1)
    return mean_emb, seq_embs, mask, batch

for param in decoder_model.parameters():
    param.requires_grad = False

In [7]:
sentences = [' a']

embs, last_hidden_state, _, _ = text_to_sonar_embedding(sentences, tokenizer, decoder_model.model.encoder, batch=None, lang="eng_Latn")
print(embs.shape)

target_lang = "eng_Latn"
lang_token = last_hidden_state.last_hidden_state[0][0].unsqueeze(0).unsqueeze(0)

del embs, last_hidden_state, _
decoder_model.to(device)

torch.Size([1, 1024])


M2M100ForConditionalGeneration(
  (model): M2M100Model(
    (shared): M2M100ScaledWordEmbedding(256206, 1024, padding_idx=1)
    (encoder): M2M100Encoder(
      (embed_tokens): M2M100ScaledWordEmbedding(256206, 1024, padding_idx=1)
      (embed_positions): M2M100SinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0-23): 24 x M2M100EncoderLayer(
          (self_attn): M2M100Attention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=8192, bias=True)
          (fc2): Linear(in_features=8192, out_features=1024, bias=True)
       

In [8]:
from huggingface_hub import snapshot_download

# dset = snapshot_download("pixparse/cc12m-wds", allow_patterns=["cc12m-train-0000.tar"], repo_type="dataset")

# dset = snapshot_download("pixparse/cc12m-wds", allow_patterns=["cc12m-train-000[0-9].tar"], repo_type="dataset")
# dset = snapshot_download("pixparse/cc12m-wds", allow_patterns=["cc12m-train-001[0-9].tar"], repo_type="dataset")
dset = snapshot_download("pixparse/cc12m-wds", allow_patterns=["cc12m-train-002[0-9].tar"], repo_type="dataset")
dset = snapshot_download("pixparse/cc12m-wds", allow_patterns=["cc12m-train-003[0-9].tar"], repo_type="dataset")
dset = snapshot_download("pixparse/cc12m-wds", allow_patterns=["cc12m-train-004[0-9].tar"], repo_type="dataset")
dset = snapshot_download("pixparse/cc12m-wds", allow_patterns=["cc12m-train-005[0-9].tar"], repo_type="dataset")
dset = snapshot_download("pixparse/cc12m-wds", allow_patterns=["cc12m-train-006[0-9].tar"], repo_type="dataset")

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

cc12m-train-0021.tar:   0%|          | 0.00/539M [00:00<?, ?B/s]

cc12m-train-0026.tar:   0%|          | 0.00/543M [00:00<?, ?B/s]

cc12m-train-0025.tar:   0%|          | 0.00/541M [00:00<?, ?B/s]

cc12m-train-0020.tar:   0%|          | 0.00/546M [00:00<?, ?B/s]

...

cc12m-train-0064.tar:   0%|          | 0.00/542M [00:00<?, ?B/s]

cc12m-train-0065.tar:   0%|          | 0.00/533M [00:00<?, ?B/s]

cc12m-train-0067.tar:   0%|          | 0.00/548M [00:00<?, ?B/s]

cc12m-train-0068.tar:   0%|          | 0.00/547M [00:00<?, ?B/s]

cc12m-train-0069.tar:   0%|          | 0.00/544M [00:00<?, ?B/s]

In [9]:
def process_img_only(batch):
    rgb_images = []
    succeed = []

    for img in batch["image"]:
        try:
            rgb_images.append(img.convert("RGB"))
            succeed.append(True)
        except Exception:
            succeed.append(False)
        
    processed = image_processor(
      images=rgb_images,
      return_tensors="pt",
    )
    
    return {
        "pixel_values": processed["pixel_values"],
        "succeed": succeed
    }

def process_img(batch):    
    rgb_images = [img.convert("RGB") for img in batch["jpg"]]
    processed_img = image_processor(images=rgb_images, return_tensors="pt")
    
    return {
        "pixel_values": processed_img["pixel_values"],
        "caption": batch["txt"]
    }

import os as _os
_hf_token = _os.getenv("HF_TOKEN")
if _hf_token:
    login(_hf_token)
else:
    print("HF_TOKEN not set; proceeding without login.")

dataset_train = load_dataset(dset, streaming=True)
dataset_train = dataset_train.map(process_img, batched=True, batch_size=32, remove_columns=list(next(iter(dataset_train.values())).features)).with_format("torch")

val_hf_dataset = load_dataset("pixparse/cc12m-wds", data_files="cc12m-train-0999.tar")
val_hf_dataset = val_hf_dataset["train"]
val_hf_dataset = val_hf_dataset.select(range(0, 1000))
val_hf_dataset = val_hf_dataset.map(process_img, batched=True, batch_size=128, remove_columns=val_hf_dataset.column_names, keep_in_memory=True)

HF_TOKEN not set; proceeding without login.


Resolving data files:   0%|          | 0/50 [00:00<?, ?it/s]

README.md: 0.00B [00:00, ?B/s]

cc12m-train-0999.tar:   0%|          | 0.00/544M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [10]:
train_hf_dataset = dataset_train["train"]
train_hf_dataset = train_hf_dataset.take(100000)
batch_size = 3
train_loader = DataLoader(train_hf_dataset, batch_size=batch_size)

val_hf_dataset.set_format("torch")
val_loader = DataLoader(val_hf_dataset, batch_size=batch_size, shuffle=False)

In [11]:
import bitsandbytes as bnb

params_to_train = [p for p in image_encoder.parameters() if p.requires_grad]

optimizer = bnb.optim.AdamW8bit(params_to_train, lr=1e-4, weight_decay=1e-2)

loss_targets = {
    'lambda_ce': 1.0,
    'lambda_nce': 0.05,
    'lambda_cos': 0.05,
    'lambda_mse': 0.0,
}

ce_warmup_steps = 2000          
align_warmup_steps = 2000 

In [12]:
num_epochs = 5
save_dir = "./adapter_checkpoints"
os.makedirs(save_dir, exist_ok=True)

In [13]:
import gc

gc.collect()

30

In [14]:
import torch
import torch.nn.functional as F
from torch import nn
from tqdm import tqdm
import os

def compute_info_nce_loss(image_emb, text_emb, logit_scale=None, temperature=None):
    image_emb = F.normalize(image_emb, dim=-1)
    text_emb = F.normalize(text_emb, dim=-1)
    logits = image_emb @ text_emb.T
    if logit_scale is not None:
        logits = logits * logit_scale
    elif temperature is not None:
        logits = logits / temperature
    labels = torch.arange(len(logits), device=logits.device)
    loss_i2t = F.cross_entropy(logits, labels)
    loss_t2i = F.cross_entropy(logits.T, labels)
    return 0.5 * (loss_i2t + loss_t2i)
    
def train_model(
    global_step=0,
    max_steps=8000,
    log_every=1000,
    eval_every=1000,
    save_every=2000,
    grad_accum_steps=16
):
    best_val_loss = float("inf")
    epoch = 0
    lang_token.to("cuda")
    image_encoder.to("cuda")
    
    while global_step < max_steps:
        image_encoder.train()
       
        decoder_model.eval()
        epoch += 1
        
        pbar = tqdm(train_loader, desc=f"Epoch {epoch} (Step {global_step})")
        optimizer.zero_grad()
        for micro_step, batch in enumerate(pbar):
            pixel_values = batch['pixel_values'].to(device)
            captions = batch['caption']

            tokenized_batch = tokenizer(
                captions, return_tensors="pt", padding=True, truncation=True
            ).to(device)
            
            labels = tokenized_batch['input_ids'].clone()
            labels[labels == tokenizer.pad_token_id] = -100
            
            image_emb_pred, logit_scale, _, _ = image_encoder(pixel_values)

            if global_step < ce_warmup_steps:
                align_scale = 0.0
            else:
                ramp = (global_step - ce_warmup_steps) / max(1, align_warmup_steps)
                align_scale = float(max(0.0, min(1.0, ramp)))

            eff_lambda_ce  = loss_targets['lambda_ce']
            eff_lambda_nce = loss_targets['lambda_nce'] * align_scale
            eff_lambda_cos = loss_targets['lambda_cos'] * align_scale
            eff_lambda_mse = loss_targets['lambda_mse'] * align_scale

            compute_align = (eff_lambda_nce > 0) or (eff_lambda_cos > 0) or (eff_lambda_mse > 0)
            if compute_align:
                text_emb_target, _, _, _ = text_to_sonar_embedding(
                    encoder_model=decoder_encoder,
                    batch=tokenized_batch,
                    lang='eng_Latn',
                    device=device
                )
          
            enc_attn_mask = torch.ones((image_emb_pred.size(0), 1), dtype=torch.long, device=device)
            enc_outputs = BaseModelOutput(last_hidden_state=image_emb_pred.unsqueeze(1))

            outputs_pred = decoder_model(
                encoder_outputs=enc_outputs,
                attention_mask=enc_attn_mask,
                labels=labels,
                return_dict=True
            )
            
            ce_loss = outputs_pred.loss

            if compute_align:
                mse_loss = F.mse_loss(F.normalize(image_emb_pred, dim=-1), F.normalize(text_emb_target, dim=-1)) if eff_lambda_mse > 0 else torch.tensor(0.0, device=device)
                cos_loss = 1 - F.cosine_similarity(image_emb_pred, text_emb_target, dim=-1).mean() if eff_lambda_cos > 0 else torch.tensor(0.0, device=device)
                nce_loss = compute_info_nce_loss(image_emb_pred, text_emb_target, logit_scale=logit_scale) if eff_lambda_nce > 0 else torch.tensor(0.0, device=device)
            else:
                mse_loss = torch.tensor(0.0, device=device)
                cos_loss = torch.tensor(0.0, device=device)
                nce_loss = torch.tensor(0.0, device=device)

            loss = (
                eff_lambda_ce  * ce_loss +
                eff_lambda_mse * mse_loss +
                eff_lambda_cos * cos_loss +
                eff_lambda_nce * nce_loss
            )

            (loss / grad_accum_steps).backward()

            should_step = ((micro_step + 1) % grad_accum_steps == 0)
            if should_step:
                train_params = []
                train_params += list(image_encoder.parameters())
   
                torch.nn.utils.clip_grad_norm_(train_params, 1.0)
                optimizer.step()
                optimizer.zero_grad()
                global_step += 1

                if global_step % log_every == 0:
                    pbar.set_postfix({
                        'step': global_step,
                        'loss': f"{loss.item():.4f}",
                        'ce'  : f"{ce_loss.item():.3f}",
                        'mse' : f"{mse_loss.item():.3f}",
                        'nce' : f"{nce_loss.item():.3f}",
                        'cos' : f"{cos_loss.item():.3f}",
                        'a-scal': f"{align_scale:.2f}",
                    })
                    print(
                        f"\nStep {global_step} | Loss: {loss.item():.4f} | CE: {ce_loss.item():.3f} | "
                        f"MSE: {mse_loss.item():.3f} | NCE: {nce_loss.item():.3f} | COS: {cos_loss.item():.3f} | ALIGN-SCALE: {align_scale:.2f}"
                    )

                if global_step % eval_every == 0:
                    val_loss = validate(
                        image_encoder, None, decoder_model, val_loader, decoder_encoder, tokenizer, device
                    )
                    print(f"\nStep {global_step} | Val Loss: {val_loss:.4f}")
                    
                    if val_loss < best_val_loss:
                        best_val_loss = val_loss
                        save_path = os.path.join(save_dir, f"best_model_step{global_step}.pt")
                        save_payload = {
                            'step': global_step,
                            'epoch': epoch,
                            'image_encoder_state_dict': image_encoder.state_dict(),
                            'optimizer_state_dict': optimizer.state_dict(),
                            'val_loss': val_loss
                        }

                        torch.save(save_payload, save_path)
                        print(f"✅ New best model saved at {save_path}")

                # Checkpoint saving
                if global_step % save_every == 0:
                    save_path = os.path.join(save_dir, f"checkpoint_step{global_step}.pt")
                    save_payload = {
                        'step': global_step,
                        'epoch': epoch,
                        'image_encoder_state_dict': image_encoder.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                    }
                
                    torch.save(save_payload, save_path)
                    print(f"💾 Checkpoint saved at {save_path}")

                if global_step >= max_steps:
                    break
        
        if global_step >= max_steps:
            break

    print(f"\nTraining complete! Reached {global_step} steps")
    print(f"Best validation loss: {best_val_loss:.4f}")
    return global_step


def validate(image_encoder, visual_prefix, decoder_model, val_loader, decoder_encoder, tokenizer, device, max_batches=None):
    image_encoder.eval()
    
    decoder_model.eval()

    total_loss, total_mse, total_ce, total_nce, total_cos = 0, 0, 0, 0, 0
    num_batches = 0

    with torch.no_grad():
        for i, batch in enumerate(val_loader):
            if max_batches and i >= max_batches:
                break
                
            pixel_values = batch['pixel_values'].to(device)
            captions = batch['caption']

            tokenized_batch = tokenizer(
                captions, return_tensors="pt", padding=True, truncation=True
            ).to(device)
            labels = tokenized_batch['input_ids'].clone()
            labels[labels == tokenizer.pad_token_id] = -100
            
            image_emb_pred, logit_scale, _, _ = image_encoder(pixel_values)

            if global_step < ce_warmup_steps:
                align_scale = 0.0
            else:
                ramp = (global_step - ce_warmup_steps) / max(1, align_warmup_steps)
                align_scale = float(max(0.0, min(1.0, ramp)))

            eff_lambda_ce  = loss_targets['lambda_ce']
            eff_lambda_nce = loss_targets['lambda_nce'] * align_scale
            eff_lambda_cos = loss_targets['lambda_cos'] * align_scale
            eff_lambda_mse = loss_targets['lambda_mse'] * align_scale
            
            compute_align = (eff_lambda_nce > 0) or (eff_lambda_cos > 0) or (eff_lambda_mse > 0)
            if compute_align:
                text_emb_target, _, _, _ = text_to_sonar_embedding(
                    encoder_model=decoder_encoder,
                    batch=tokenized_batch,
                    lang='eng_Latn',
                    device=device
                )

          
            enc_attn_mask = torch.ones((image_emb_pred.size(0), 1), dtype=torch.long, device=device)
            enc_outputs = BaseModelOutput(last_hidden_state=image_emb_pred.unsqueeze(1))
            
            outputs_pred = decoder_model(
                encoder_outputs=enc_outputs,
                attention_mask=enc_attn_mask,
                labels=labels,
                return_dict=True
            )
            
            ce_loss = outputs_pred.loss

            if compute_align:
                mse_loss = F.mse_loss(F.normalize(image_emb_pred, dim=-1), F.normalize(text_emb_target, dim=-1)) if eff_lambda_mse > 0 else torch.tensor(0.0, device=device)
                cos_loss = 1 - F.cosine_similarity(image_emb_pred, text_emb_target, dim=-1).mean() if eff_lambda_cos > 0 else torch.tensor(0.0, device=device)
                nce_loss = compute_info_nce_loss(image_emb_pred, text_emb_target, logit_scale=logit_scale) if eff_lambda_nce > 0 else torch.tensor(0.0, device=device)
            else:
                mse_loss = torch.tensor(0.0, device=device)
                cos_loss = torch.tensor(0.0, device=device)
                nce_loss = torch.tensor(0.0, device=device)

            loss = (
                eff_lambda_ce  * ce_loss +
                eff_lambda_mse * mse_loss +
                eff_lambda_cos * cos_loss +
                eff_lambda_nce * nce_loss
            )

            total_loss += loss.item()
            total_mse += mse_loss.item()
            total_ce += ce_loss.item()
            total_nce += nce_loss.item()
            total_cos += cos_loss.item()
            num_batches += 1

            if i % 50 == 0:
                print(f"'loss': {loss.item():.4f},'ce': {ce_loss.item():.3f},'mse': {mse_loss.item():.3f},'nce': {nce_loss.item():.3f},cos': {cos_loss.item():.3f}")
                
                enc_for_gen = enc_outputs
                
                generated_tokens_en = decoder_model.generate(
                    encoder_outputs=enc_for_gen,
                    attention_mask=enc_attn_mask,
                    forced_bos_token_id=tokenizer.convert_tokens_to_ids('eng_Latn'),
                    max_new_tokens=32,
                )

                generated_texts_en = tokenizer.batch_decode(generated_tokens_en, skip_special_tokens=True)
                tokenizer.tgt_lang = 'eng_Latn'
                tokenizer.src_lang = 'ben_Beng'
                
                generated_tokens_bn = decoder_model.generate(
                    encoder_outputs=enc_for_gen,
                    attention_mask=enc_attn_mask,
                    forced_bos_token_id=tokenizer.convert_tokens_to_ids('ben_Beng'),
                    max_new_tokens=32,
                )

                generated_texts_bn = tokenizer.batch_decode(generated_tokens_bn, skip_special_tokens=True)
                tokenizer.tgt_lang = 'eng_Latn'
                tokenizer.src_lang = 'eng_Latn'
                
                print("Generated En:", generated_texts_en[:2])
                print("Generated Bn:", generated_texts_bn[:2])
                print("True:", captions[:2])

    avg_loss = total_loss / num_batches if num_batches > 0 else 0
    print (
        f"Validation | Avg Loss: {avg_loss:.4f} | CE: {total_ce/num_batches:.4f} | "
        f"MSE: {total_mse/num_batches:.4f} | NCE: {total_nce/num_batches:.4f} | COS: {total_cos/num_batches:.4f}"
    )

    return avg_loss

In [15]:
global_step = 0
train_model(global_step)

Epoch 1 (Step 0): 0it [00:00, ?it/s]/usr/local/lib/python3.11/dist-packages/datasets/formatting/torch_formatter.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(value, **{**default_dtype, **self.torch_tensor_kwargs})
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.58.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
Epoch 1 (Step 0): 1984it [07:19,  5.78it/s]/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))
Epoch 1 (Step 0): 2517it [09:15,  6.00it/s]/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.wa


Step 1000 | Loss: 3.9775 | CE: 3.977 | MSE: 0.000 | NCE: 0.000 | COS: 0.000 | ALIGN-SCALE: 0.00
'loss': 3.9399,'ce': 3.940,'mse': 0.000,'nce': 0.000,cos': 0.000
Generated En: ['The building where the hotel is located', 'The couple on the beach with a wedding dress']
Generated Bn: ['একটি ভবন যেখানে একটি ভবন ছিল', 'বিবাহিত দম্পতি এবং একটি বিবাহিত দম্পতি']
True: ['The Living Room at the W Midtown in Atlanta', 'Destination Wedding in Tulum | The Celebration Society']
'loss': 3.1815,'ce': 3.182,'mse': 0.000,'nce': 0.000,cos': 0.000
Generated En: ["The 'PERSON' is a beautiful, beautiful, beautiful, beautiful, beautiful, beautiful, beautiful, beautiful, beautiful, beautiful, beautiful, beautiful", 'A text message from a user to a user']
Generated Bn: ['<PERSON>, <PERSON>, <PERSON>, <PERSON>, <PERSON>, <', 'একটি প্যারিশের একটি টুকরা']
True: ['<PERSON>...made the <PERSON> eye her own. Nobody does it quite like <PERSON>. Love... <PERSON>...made the <PERSON> eye her own. Nobody does it quite lik

Epoch 1 (Step 0): 16000it [1:00:06, 20.94s/it, step=1000, loss=3.9775, ce=3.977, mse=0.000, nce=0.000, cos=0.000, a-scal=0.00]

✅ New best model saved at ./adapter_checkpoints/best_model_step1000.pt


Epoch 1 (Step 0): 31999it [1:59:05,  5.44it/s, step=2000, loss=3.5799, ce=3.580, mse=0.000, nce=0.000, cos=0.000, a-scal=0.00]

...

Epoch 4 (Step 6249): 28015it [2:01:45,  4.16it/s, step=8000, loss=3.7458, ce=3.695, mse=0.000, nce=0.158, cos=0.848, a-scal=1.00]


Step 8000 | Loss: 3.7458 | CE: 3.695 | MSE: 0.000 | NCE: 0.158 | COS: 0.848 | ALIGN-SCALE: 1.00
'loss': 3.7585,'ce': 3.759,'mse': 0.000,'nce': 0.000,cos': 0.000
Generated En: ['The lobby, or lobby, in the hotel', 'Wedding Photography - The Wedding Photography of a Couple']
Generated Bn: ['লন্ডন, লন্ডন, লন্ডন, লন্ডন, লন্ডন, লন্ডন, লন্ডন, লন্ডন', '<PERSON>, <PERSON>, <PERSON>, <PERSON>, <PERSON>, <']
True: ['The Living Room at the W Midtown in Atlanta', 'Destination Wedding in Tulum | The Celebration Society']
'loss': 3.0214,'ce': 3.021,'mse': 0.000,'nce': 0.000,cos': 0.000
Generated En: ['<PERS>, <PERS>, <PERS>, <PERS>, <PERS>, <PERS>, <', 'Texting on a phone app app']
Generated Bn: ['<PERS>, <PERS>, <PERS>, <PERS>, <PERS>, <PERS>, <', '<PERSON> ট্যাবলেটে একটি স্পোন ব্যবহার করে']
True: ['<PERSON>...made the <PERSON> eye her own. Nobody does it quite like <PERSON>. Love... <PERSON>...made the <PERSON> eye her own. Nobody does it quite like <PERSON>. Vintage Makeup, Vintage Beauty, 1960s

Epoch 4 (Step 6249): 28015it [2:02:55,  3.80it/s, step=8000, loss=3.7458, ce=3.695, mse=0.000, nce=0.158, cos=0.848, a-scal=1.00]

💾 Checkpoint saved at ./adapter_checkpoints/checkpoint_step8000.pt

Training complete! Reached 8000 steps
Best validation loss: 3.4948


8000